In [2]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from torch.utils.data import Dataset, DataLoader
import clearml
from clearml import Task, Logger

In [6]:
df_1 = pd.read_csv('C://Users//milai/Downloads/data2.csv')
df = df_1
df

,sentiment_score,message
0,0.739,Все на питоне
1,0.245,"Ребят, а как хантеры относятся к начинающим ст..."
2,-0.234,"Ни разу не сказали, что берут для повышения оц..."
3,0.152,"А прям точно нужна кластеризация? Возможно, пр..."
4,0.120,"на расчет метрик, придумываете новых, на аб тесты"
...,...,...
29995,0.500,это норм. фильтруется таблица и после этого то...
29996,0.200,а это вы на ml-инженера или на data аналитика ...
29997,-0.300,lms по этой же причине не отрывается?
29998,-0.125,вообще я думал через and написать и в date_par...


In [7]:
class JoyoMeterDataset(Dataset):
    def __init__(self, features, target):
        self.features = features
        self.target = target

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index): 
        feature = self.features[index]
        target = self.target[index]
        return feature, target

In [8]:
X = df['message'].reset_index(drop=True)
y = df['sentiment_score'].reset_index(drop=True)

# Правильное разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

train_dataset = JoyoMeterDataset(X_train, y_train)
val_dataset = JoyoMeterDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from joblib import Memory

memory = Memory("cache_dir", verbose=0)


class BertRegression(nn.Module):
    def __init__(self,
                 model_name="DeepPavlov/rubert-base-cased",
                 pretrained_weights_path="destination_filename.pth"):
        super(BertRegression, self).__init__()
        
        torch.manual_seed(50) 

        self.bert = AutoModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.bert.to(torch.device('cpu'))
        self.dropout = nn.Dropout(0.2)
        self.layer_norm = nn.LayerNorm(self.bert.config.hidden_size)
        self.regressor = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 256), nn.Tanh(),
            nn.Linear(256, 1))

        self.load_state_dict(
            torch.load(pretrained_weights_path,
                       map_location=torch.device('cpu'),
                       weights_only=True))

        self.device = 'cpu'
        self.to(self.device)

    @memory.cache(ignore=['self'])
    def tokenize(self, input_text):
        return self.tokenizer(input_text,
                              return_tensors="pt",
                              max_length=512,
                              truncation=True,
                              padding='max_length')

    def predict(self, input_text):
        with torch.no_grad():
            tokens = self.tokenize(self, input_text)
            input_ids = tokens["input_ids"].to(self.device)
            attention_mask = tokens["attention_mask"].to(self.device)
            outputs = self.bert(input_ids=input_ids,
                                attention_mask=attention_mask)
            pooled_output = outputs.pooler_output
            dropout_output = self.dropout(pooled_output)
            layer_norm_output = self.layer_norm(dropout_output)
            linear_output = self.regressor(layer_norm_output)
            return linear_output.item()

In [21]:
model = BertRegression(device='cpu')
epochs = 5
accumulation_steps = 4
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(len(train_dataloader))
print(len(val_dataloader))

In [9]:
bert_regression_model.load_state_dict(torch.load('bertweight2new2.pth'))
bert_regression_model.to(device)
import torch
import torch.nn as nn
device = 'cuda'
def predict(text, model, tokenizer, device):
  """
  Функция для предсказания модели BERT для заданного текста.

  Args:
      text: Текст для предсказания.
      model: Модель BERT (BertRegression).
      tokenizer: Токенизатор для BERT.
      device: Устройство для вычислений (CPU или GPU).

  Returns:
      Предсказанное непрерывное значение.
  """

  # Токенизация текста
  inputs = tokenizer_deep(text, return_tensors="pt", add_special_tokens=True, padding=True).to(device)

  # Вывод модели
  with torch.no_grad():
    outputs = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
    )

  # Получение предсказания
  output = outputs.squeeze().item()  # Извлекаем выход из последнего слоя (полносвязный)
  return output

# Пример использования:
text =  "да ебать вас"
predicted_value = predict(text, bert_regression_model, tokenizer_deep, device) 
print(f"Predicted value: {predicted_value}")

Predicted value: -0.38380342721939087


D:\MY_PROGRAMMIER_SLOTS\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [7]:
device = 'cuda'

def calculate_rmse(outputs, scores, criterion):
    """Вычисляет RMSE для заданных выходов и меток."""
    rmse = torch.sqrt(criterion(outputs.squeeze(), scores.to(device).float())).mean()
    return rmse.item()

def train_step(model, optimizer, criterion, dataloader, epoch, device):
    """Обучение модели на одной эпохе."""
    model.train()
    train_loss = 0.0
    train_rmse = 0.0
    train_len = len(dataloader)
    for tweets, scores in tqdm(dataloader, total=train_len, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
        inputs = tokenizer_deep.batch_encode_plus(
            tweets,
            return_tensors='pt',
            add_special_tokens=True,
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        # Получение предсказаний
        outputs = model(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)

        # Вычисление потерь и обратное распространение
        loss = criterion(outputs.squeeze(), scores.to(device).float()).to(device)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        train_rmse += calculate_rmse(outputs, scores, criterion)

    train_loss_avg = train_loss / train_len
    train_rmse_avg = train_rmse / train_len
    return train_loss_avg, train_rmse_avg

def validation_step(model, criterion, dataloader, epoch, device):
    """Оценка модели на валидационных данных."""
    model.eval()
    val_loss = 0
    val_rmse = 0
    val_len = len(dataloader)

    with torch.no_grad():
        for tweets, scores in tqdm(dataloader, total=val_len, desc=f"Validation", leave=False):
            inputs = tokenizer_deep.batch_encode_plus(
                tweets,
                return_tensors='pt',
                add_special_tokens=True,
                padding=True,
                truncation=True,
                max_length=512
            ).to(device)

            # Получение предсказаний
            outputs = model(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)

            # Вычисление потерь
            val_loss += criterion(outputs.squeeze(), scores.to(device).float()).item()
            val_rmse += calculate_rmse(outputs, scores, criterion)

    val_loss_avg = val_loss / val_len
    val_rmse_avg = val_rmse / val_len
    return val_loss_avg, val_rmse_avg

task = Task.init(project_name='Bertv2', task_name='BertTrain')
logger = Logger.current_logger()
best_val_loss = float('inf')

# Цикл обучения
for epoch in range(epochs):
    train_loss, train_rmse = train_step(bert_regression_model, optimizer, criterion, train_dataloader, epoch, device)
    val_loss, val_rmse = validation_step(bert_regression_model, criterion, val_dataloader, epoch, device)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.3f}, Train RMSE: {train_rmse:.3f}, Val Loss: {val_loss:.3f}, Val RMSE: {val_rmse:.3f}")

    # Логирование метрик в ClearML
    logger.report_scalar(title='Train Loss', series='loss', value=train_loss, iteration=epoch)
    logger.report_scalar(title='Train RMSE', series='rmse', value=train_rmse, iteration=epoch)
    logger.report_scalar(title='Validation Loss', series='loss', value=val_loss, iteration=epoch)
    logger.report_scalar(title='Validation RMSE', series='rmse', value=val_rmse, iteration=epoch)

    # Сохранение модели
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(bert_regression_model.state_dict(), 'bertweight2new2.pth')
    if epoch == epochs - 1:  # Проверяем,  последняя ли это эпоха
        torch.save(bert_regression_model.state_dict(), 'bertweight2new2last.pth')
        task.upload_artifact('model', 'bertweight2new2last.pth')

# Завершение задачи ClearML
task.close()

ClearML Task: created new task id=3d64ef93274d49da830909d8f397d399
ClearML results page: https://app.clear.ml/projects/f857a6e643a74a039cefea4c76612313/experiments/3d64ef93274d49da830909d8f397d399/output/log
2024-08-30 22:23:34,445 - clearml.Task - INFO - Storing jupyter notebook directly as code


Epoch 1/5:   0%|                                                                               | 0/410 [00:00<?, ?it/s]D:\MY_PROGRAMMIER_SLOTS\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Epoch 1/5:  38%|██████████████████████████▍                                          | 157/410 [02:43<02:17,  1.85it/s]

ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


Epoch 1, Train Loss: 0.108, Train RMSE: 0.323, Val Loss: 0.093, Val RMSE: 0.293
ClearML Monitor: Reporting detected, reverting back to iteration based reporting
2024-08-30 22:33:43,459 - clearml.frameworks - INFO - Found existing registered model id=e7794abf28814d84835d3ff98a911e80 [C:\Users\milai\bertweight2new2.pth] reusing it.


Epoch 2, Train Loss: 0.079, Train RMSE: 0.275, Val Loss: 0.093, Val RMSE: 0.296


Epoch 3, Train Loss: 0.059, Train RMSE: 0.238, Val Loss: 0.081, Val RMSE: 0.277


Epoch 4, Train Loss: 0.043, Train RMSE: 0.202, Val Loss: 0.084, Val RMSE: 0.280


Epoch 5, Train Loss: 0.035, Train RMSE: 0.181, Val Loss: 0.101, Val RMSE: 0.310


██████████████████████████████ 100% | 679.30/679.3 MB [29:00<00:00,  2.56s/MB]: D:\MY_PROGRAMMIER_SLOTS\anaconda3\Lib\site-packages\tqdm\std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
██████████████████████████████ 100% | 679.30/679.3 MB [29:09<00:00,  2.57s/MB]: 


In [ ]:
tqdm()